In [1]:
from seeq import spy
import pandas as pd

In [1]:
# Log into Seeq Server if you're not using Seeq Data Lab:
spy.login(url='http://localhost:34216', credentials_file='../credentials.key', force=False)

# spy.workbooks

The `spy.workbooks` module provides functions for importing and exporting _workbooks_. A workbook is either a _Workbench Analysis_ (colored green in the Seeq user interface) or an _Organizer Topic_ (colored blue).

This functionality is helpful to:

- Move content between two Seeq servers
- Manage content by exporting and committing to a version control system like Git

The process typically starts by searching for some content that you have created in Seeq and exporting it. However, since this documentation needs some pre-built content to illustrate how it works, there is a pre-built-and-exported set of workbooks alongside this documentation notebook. So we'll go in a non-typical order of operations in this example.

## The Export Format

When content is exported from Seeq, each workbook is encapsulated in its own folder, including all its worksheets, calculated item definitions and all dependencies, journal/document images and anything else that is necessary to import that data into another server. Content is written to disk as either JSON, HTML or image files as appropriate. References to datasource items are also catalogued during export and default _datasource maps_ are created that facilitate identification of equivalent signals/conditions/scalars on the destination system so that the imported content gets "hooked up" to the right data.

## Main Actions

There are five main operations you can perform on workbooks:

- **search** for workbooks whose content you want to *pull*
- **pull** those workbooks into `Workbook` in-memory Python objects
- **save** the `Workbook` Python objects to disk in the export format described above
- **load** `Workbook` Python objects from disk into memory
- **push** in-memory `Workbook` Python objects into a Seeq Server

As mentioned, we're going to go out-of-order for illustration purposes: _load_, _push_, _search_, _pull_, _save_.

### Importing

This set of documentation comes with an _Example Export_ folder that contains an Analysis and a Topic for illustration purposes. First we **load** it into memory:

In [3]:
workbooks = spy.workbooks.load('Example Export.zip')
workbooks

[Workbook "Example Analysis" (D833DC83-9A38-48DE-BF45-EB787E9E8375),
 Workbook "Example Topic" (811B1488-297A-4FD2-AE7C-A1FE0E3B3641)]

Now that the workbook definitions are in memory, we can push them into Seeq.

In [4]:
spy.workbooks.push(workbooks, path='My Import', errors='raise')

,ID,Name,Type,Workbook Type,Count,Time,Result,Pushed Workbook ID,URL
0,D833DC83-9A38-48DE-BF45-EB787E9E8375,Example Analysis,Workbook,Analysis,54,00:00:02.39,Success,7A7F7AC7-F68E-42E3-9F76-0BCEC34521F3,link
1,811B1488-297A-4FD2-AE7C-A1FE0E3B3641,Example Topic,Workbook,Topic,6,00:00:00.28,Success,9D64A960-AA1A-43DC-BACD-E580FFBA8F70,link


,ID,Name,Type,Workbook Type,Count,Time,Result,Pushed Workbook ID,URL
0,D833DC83-9A38-48DE-BF45-EB787E9E8375,Example Analysis,Workbook,Analysis,54,0:00:02.393995,Success,7A7F7AC7-F68E-42E3-9F76-0BCEC34521F3,http://localhost:34216/3BEDBD8B-522D-4063-807B...
1,811B1488-297A-4FD2-AE7C-A1FE0E3B3641,Example Topic,Workbook,Topic,6,0:00:00.276998,Success,9D64A960-AA1A-43DC-BACD-E580FFBA8F70,http://localhost:34216/3BEDBD8B-522D-4063-807B...


The workbooks have been imported into Seeq in a _My Import_ folder with you as the owner. Refresh Seeq Workbench in your browser and take a look.

### Exporting

In Seeq Workbench, try changing the name of the _Example Analysis_ workbook to something like _My First Analysis Export_ so that you can tell that your changes get exported.

Now we will **search** for the workbooks we want to export. The syntax for a workbook search query is very similar to an item metadata search via `spy.search()`:

In [5]:
workbooks_df = spy.workbooks.search({
    'Path': 'My Import'
})

workbooks_df

,Count,Time
Results,2,00:00:00.08


,Access Level,Created At,ID,Name,Owner ID,Owner Name,Owner Username,Path,Pinned,Search Folder ID,Type,Updated At,Workbook Type
0,FULL_CONTROL,2020-01-06 06:00:30.866565+00:00,7A7F7AC7-F68E-42E3-9F76-0BCEC34521F3,Example Analysis,F9096D91-B9F3-4907-9D0A-DC792160B058,Agent API Key,agent_api_key,My Import,0.0,3BEDBD8B-522D-4063-807B-3EA4A795CE63,Workbook,2020-01-06 22:12:45.052986+00:00,Analysis
1,FULL_CONTROL,2020-01-06 06:00:33.203560900+00:00,9D64A960-AA1A-43DC-BACD-E580FFBA8F70,Example Topic,F9096D91-B9F3-4907-9D0A-DC792160B058,Agent API Key,agent_api_key,My Import,0.0,3BEDBD8B-522D-4063-807B-3EA4A795CE63,Workbook,2020-01-06 22:12:45.501982900+00:00,Topic


As you can see, the `spy.workbooks.search()` command returns a metadata DataFrame with the properties of the workbooks. We can now use that to **pull**:

In [6]:
workbooks = spy.workbooks.pull(workbooks_df)
workbooks

,ID,Name,Workbook Type,Count,Time,Result
0,7A7F7AC7-F68E-42E3-9F76-0BCEC34521F3,Example Analysis,Analysis,53,00:00:02.18,Success
1,9D64A960-AA1A-43DC-BACD-E580FFBA8F70,Example Topic,Topic,6,00:00:00.35,Success


[Workbook "Example Analysis" (7A7F7AC7-F68E-42E3-9F76-0BCEC34521F3),
 Workbook "Example Topic" (9D64A960-AA1A-43DC-BACD-E580FFBA8F70)]

These are the same type of in-memory Python objects that we had when we executed `spy.workbooks.load()`. Now we can **save** them to disk:

In [7]:
spy.workbooks.save(workbooks, '../My First Export', clean=True)

In the parent folder of this documentation notebook, you'll find a new _My First Export_ folder that contains similar files to the _Example Export_ folder that's part of the documentation.

## Inspecting Worksheets

With the in-memory Python objects that result from `spy.workbooks.pull()` or `spy.workbooks.load()`, you can inspect the worksheets to see what is displayed on them. For example, let's look at what's in the Details Pane of the second worksheet of _Example Analysis_:

In [8]:
worksheet_items = workbooks[0].worksheets[1].display_items
worksheet_items

,Axis Align,Axis Auto Scale,Axis Group,Axis Max,Axis Min,Axis Show,Color,ID,Lane,Line Style,Line Width,Name,Samples Display,Type
0,Left,True,A,120.433655,66.778460,True,#068C45,366C2FB1-684C-4380-8889-EE1B6E3A93FE,2.0,Solid,1.0,Area A_Temperature,Line,Signal
1,Left,True,A,120.433655,66.778460,True,#9D248F,AC584335-F2DA-4D36-8C23-FDFA8B1A0B17,2.0,Solid,2.5,Smooth Temperature,Line,Signal
2,Left,True,C,38.162385,0.002923,True,#CE561B,CF189BEB-7CAD-4D18-98FD-3624AB29AC14,1.0,Solid,1.0,Area A_Compressor Power,Line,Signal
3,Left,True,D,96.598636,-100.000000,True,#00A2DD,CF1D50FE-3FC1-4C42-A04A-024D6F1BA83B,3.0,Solid,1.0,Area C_Temperature,Line,Signal
4,NaN,NaN,NaN,NaN,NaN,NaN,#4055A3,29607AA0-E6A2-43CC-9076-590312517875,NaN,NaN,NaN,High Power,NaN,Condition
5,Left,True,A,120.433655,66.778460,True,#E1498E,FE96FF07-06FB-40AA-B136-99498375AE72,2.0,Dash,1.0,Temperature Limit,NaN,Scalar


Now you can call `spy.pull()` to pull data for the items in the worksheet.

In [9]:
spy.pull(worksheet_items, start='2019-01-01T00:00:00', end='2019-01-02T00:00:00')

,ID,Name,Count,Time,Result
0,366C2FB1-684C-4380-8889-EE1B6E3A93FE,Area A_Temperature,97,00:00:00.08,Success
1,AC584335-F2DA-4D36-8C23-FDFA8B1A0B17,Smooth Temperature,97,00:00:00.25,Success
2,CF189BEB-7CAD-4D18-98FD-3624AB29AC14,Area A_Compressor Power,97,00:00:00.07,Success
3,CF1D50FE-3FC1-4C42-A04A-024D6F1BA83B,Area C_Temperature,97,00:00:00.08,Success
4,29607AA0-E6A2-43CC-9076-590312517875,High Power,1,00:00:00.31,Success
5,FE96FF07-06FB-40AA-B136-99498375AE72,Temperature Limit,1,00:00:00.17,Success


,Area A_Temperature,Smooth Temperature,Area A_Compressor Power,Area C_Temperature,High Power,Temperature Limit
2019-01-01 00:00:00+00:00,93.861500,95.094408,39.189300,102.017778,1.0,120.433655
2019-01-01 00:15:00+00:00,95.364050,95.257065,39.842881,89.474636,1.0,120.433655
2019-01-01 00:30:00+00:00,95.056341,95.548184,39.949065,87.157081,1.0,120.433655
2019-01-01 00:45:00+00:00,96.938340,96.005686,39.424393,83.497438,1.0,120.433655
2019-01-01 01:00:00+00:00,97.057959,96.521943,39.326496,82.098203,1.0,120.433655
2019-01-01 01:15:00+00:00,96.516522,96.940305,39.776346,80.415390,1.0,120.433655
2019-01-01 01:30:00+00:00,97.894497,97.165759,39.977703,80.017814,1.0,120.433655
2019-01-01 01:45:00+00:00,97.179620,97.208761,40.405734,79.536580,1.0,120.433655
2019-01-01 02:00:00+00:00,97.858900,97.155982,40.223175,79.100261,1.0,120.433655
2019-01-01 02:15:00+00:00,97.564234,97.089436,39.699542,78.572570,1.0,120.433655


Note that if you just wanted the full metadata for the items, you could execute `spy.search(worksheet_items[['ID']])`.

## Re-importing and Labels

If you **push** a set of workbooks more than once, then by default you will simply overwrite the existing workbooks with the saved content. This can be useful when you are "backing up" content to disk, perhaps for the purposes of version control.

You can choose to **push** and supply a _label_, which will create a separate copy of all of the imported items instead of modifying the existing ones. This is useful when you want to import something that you are iterating on prior to affecting the "published" version. For example, let's push our workbooks with the label of `In Development`:

In [10]:
spy.workbooks.push(workbooks, path='My Development Folder', label='In Development')

,ID,Name,Type,Workbook Type,Count,Time,Result,Pushed Workbook ID,URL
0,7A7F7AC7-F68E-42E3-9F76-0BCEC34521F3,Example Analysis,Workbook,Analysis,58,00:00:02.02,Success,EB27DA5F-67EE-4536-A718-66DA867C6E75,link
1,9D64A960-AA1A-43DC-BACD-E580FFBA8F70,Example Topic,Workbook,Topic,6,00:00:00.23,Success,DBB6B706-2374-4677-A776-FFC5C25D2FBA,link


,ID,Name,Type,Workbook Type,Count,Time,Result,Pushed Workbook ID,URL
0,7A7F7AC7-F68E-42E3-9F76-0BCEC34521F3,Example Analysis,Workbook,Analysis,58,0:00:02.023999,Success,EB27DA5F-67EE-4536-A718-66DA867C6E75,http://localhost:34216/1C704FFF-F180-4CD0-8CE7...
1,9D64A960-AA1A-43DC-BACD-E580FFBA8F70,Example Topic,Workbook,Topic,6,0:00:00.225995,Success,DBB6B706-2374-4677-A776-FFC5C25D2FBA,http://localhost:34216/1C704FFF-F180-4CD0-8CE7...


If you refresh Seeq Workbench, you'll notice that there is now a _My Development Folder_ and a separate copy of the Topic and Analysis that is independent of the original -- including all calculated items.

Pushing with the same value for the `label` argument will overwrite the content for that label. Change the label again if you want yet another separate copy.


## Importing to a Different Seeq Server

You may wish to copy content to a new/different Seeq Server by exporting and then importing. For example, you might have a _development_ server where you iterate on content and a _production_ server that you publish to when finished.

In order to accomplish this, you'll do one of two actions:

- If you're using the SPy module within Seeq Data Lab, you'll copy the exported folder to the other version of Seeq Data Lab and then push it from there.
- If you're using the SPy module with your own Python set up, you'll log in to the other server and push it.